# Exercise 1: Frozen Lake

For the following exercises, we will use [Gymnasium](https://gymnasium.farama.org/), a toolkit for developing and comparing reinforcement learning algorithms. It provides a wide variety of environments, including classic control tasks, Atari games, and robotics simulations. We will start with the **Frozen Lake** problem. You can read more about this environment problem from the documentation [FrozenLake](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).

<p align="center"><img src="./images/frozen_lake.png" alt="drawing" width="400"/></p>

Some information of the environment:

- The **environment** is an 8×8 frozen grid, giving us a total of 64 states. We start in the top-left corner, and our goal—the keys—is waiting in the bottom-right corner.

- We can move in four directions: **left** (action 0), **down** (action 1), **right** (action 2), and **up** (action 3). If we try to step outside the boundaries of the grid, we simply stay in place (no falling off the map here).

- But not all tiles are safe. Some of them hide **holes** in the ice. Step into one, and it’s game over. Our job is to navigate to the goal without falling in.

- Reaching the goal gives us a reward of +1. Every other move worth zero. So we need to find a smart way to reach the goal efficiently and safely.

- The ice can be slippery, which means that we will move in the desired direction with probability $1/3$. With probability $2/3$ we will move in a direction perpendicular to our intended direction (each with probability $1/3$).


In [1]:
import os

import gymnasium as gym
import imageio
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Video

# We use these functions to plot the state values and the policy
from utils import plot_values, plot_policy_and_values

We will load the environment and set the variable `is_slippery` to `True` to make the environment slippery. This means that the agent will slip on the ice, making it more challenging to navigate the frozen lake.
You can come back later and see how results change when you set `is_slippery` to `False` to see how transition dynamics change the optimal policy to follow.

In [ ]:
is_slippery = True  # Set to False for a deterministic environment

# Here we define the environment for the FrozenLake task.
dict_env = {'id': 'FrozenLake-v1',
            'map_name': '8x8',
            'render_mode': 'human',
            'is_slippery': is_slippery,
            'render_mode': 'rgb_array'
            }

env = gym.make(**dict_env)

Let's check some variables of the environment by inspecting the `env` object:

In [ ]:
# Let's check the number of states and actions in the environment.
states = env.observation_space.n
actions = env.action_space.n
print(f'States: {states}, Actions: {actions}')

We will set the seed for reproducibility and initialize the environment:

In [ ]:
# Set a seed for reproducibility
seed = 1234

In [ ]:
# This is the initial state of the environment.
observation, info = env.reset(seed=seed)
env.action_space.seed(seed)
print(f'Initial observation: {observation}')
print(f'Initial info: {info}')

The initial observation is the state we state at (top-left corner). The probability of starting from this state is 1. Now, Let's run one episode following a random policy to see how the agent performs.



In [ ]:
done = False
total_reward = 0
frames = []

while not done:
    # Save the frame for the video
    frame = env.render()
    frames.append(frame)

    # Sample a random action from the action space
    action = env.action_space.sample()
    
    # Take a step in the environment
    observation, reward, terminated, truncated, info = env.step(action)
    
    # Add the reward to the total reward
    total_reward += reward

    # Check if the episode has ended
    done = terminated or truncated

frame = env.render()
frames.append(frame)

print("Episode finished. Total reward:", total_reward)
env.close()

In [ ]:
if not os.path.exists('videos'):
    os.makedirs('videos')

In [ ]:
random_video_path = os.path.join('videos', f'frozenlake_random_slippery_{is_slippery}.mp4')
imageio.mimsave(random_video_path, frames, fps=5)

In [ ]:
Video(random_video_path, width=400)

Well, the random agent didn’t do very well, but that’s expected. It’s simply taking random actions without any strategy. Let’s try to come up with a smarter policy to help us to get to the goal!

Remember from the lecture about the **Bellman expectation** can be written as:

$$
v_\pi (s) = \sum_{a \in A}  \pi(a | s) \sum_{s' \in S} \sum_{r \in R}P(s', r | s, a)[r + \gamma v_\pi (s')],
$$

since rewards are deterministic we can simplify the expression to:

$$
v_\pi (s) = \sum_{a \in A}  \pi(a | s) \sum_{s' \in S} P(s'| s, a)[R(s, a, s') + \gamma v_\pi (s')],
$$

### Exercise 1.1: Implement policy evaluation

The first exercise consists in implementing a policy evaluation algorithm to estimate the value function of a given policy. 

We will use an iterative method, to evaluate our policy. The idea is the following: we will calculate the value function sequentially for each state. Once we finish we will repeat this step until convergence. Since the Bellman equations are convergent mappings, we are guaranteed to converge. We could alternative solve the linear system of equations.

We need to fill the following function that takes a policy and an env object, together with a discount factor and maximum number of iterations and it returns the value function for the different states.

In [ ]:
def policy_evaluation(policy, env, discount_factor=0.9, max_iterations=1000, theta=1e-6):
    P = env.unwrapped.P
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    V = np.zeros(n_states)

    for iteration in range(max_iterations):
        delta = 0
        new_V = np.copy(V)
        for s in range(n_states):
            v = 0
            for a in range(n_actions):
                for prob, next_state, reward, _ in P[s][a]:
                    raise NotImplementedError("Comment out this line and implement the policy evaluation algorithm.")
                    v += ...
            new_V[s] = v
            delta = max(delta, abs(V[s] - v))

        V = new_V

        if delta < theta:
            print(f"Converged after {iteration + 1} sweeps.")
            break

    return V

<details>
<summary>Double click to see the solution.</summary>

```python

def policy_evaluation(policy, env, discount_factor=0.9, max_iterations=1000, theta=1e-6):
    P = env.unwrapped.P
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    V = np.zeros(n_states)

    for iteration in range(max_iterations):
        delta = 0
        new_V = np.copy(V)
        for s in range(n_states):
            v = 0
            for a in range(n_actions):
                for prob, next_state, reward, _ in P[s][a]:
                    v += policy[s, a] * prob * (reward + discount_factor * V[next_state])
                    
            new_V[s] = v
            delta = max(delta, abs(V[s] - v))

        V = new_V

        if delta < theta:
            print(f"Converged after {iteration + 1} sweeps.")
            break

    return V
```

Let's evaluate a random policy to see how the value function looks like. We will use a uniform random policy, where each action has equal probability in each state.

In [ ]:
# Policy is a matrix where each row corresponds to a state and each column corresponds to an action.
# The value at policy[s, a] is the probability of taking action a in state s.
policy = np.ones((states, actions)) / actions

# print(policy)

In [ ]:
V_pi = policy_evaluation(policy, env.env)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7, 7), constrained_layout=True)

ax[0].imshow(frames[0])
ax[0].axis('off')
plot_values(V_pi, env.env, ax=ax[1])
plt.show()

### Exercise 1.2 Implement policy improvement

Our goal is now to obtain a better policy by improving the current one using the value function we just computed. The policy improvement step involves updating the policy to choose actions that maximize the expected value based on the current value function.

To do so, we will use the state values we found, and for each state, we will update the policy to choose the action that maximizes the expected value. This is done by iterating over all states and actions, calculating the expected value for each action, and then updating the policy to favor the action with the highest expected value.

$$
q_\pi (s, a) = \sum_{s' \in S} \sum_{r \in R} P(s', r | s, a) (r + \gamma \sum_{a' \in A} \pi(a'|s')v(s'))
$$

again, since the rewards are deterministic:

$$
q_\pi (s, a) = \sum_{s' \in S} P(s'| s, a) (R(s, a, s') + \gamma \sum_{a' \in A} \pi(a'|s')v(s'))
$$

Then, we can find a new policy:

$$
\pi' (s) = \arg \max_a q_\pi(s, a)
$$

We repeat these steps until the policy stops changing (i.e., it converges). Each update improves the policy, so we get a sequence:

$$
\pi_0 < \pi_1 < ... < \pi_N = \pi^*,
$$

where $\pi^*$ is the **optimal policy**.



In [ ]:
def policy_improvement(V, env, discount_factor=0.9):
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    P = env.unwrapped.P

    policy = np.zeros((num_states, num_actions))

    for s in range(num_states):
        action_values = np.zeros(num_actions)

        for a in range(num_actions):
            for p, s_, r, _ in P[s][a]:
                raise NotImplementedError("Comment out this line and implement the policy evaluation algorithm.")
                action_values[a] += ...

        # Greedy action (or break ties arbitrarily)
        best_action = np.argmax(action_values)

        # Deterministic policy: all prob on best_action
        policy[s, best_action] = 1.0

    return policy

<details>
<summary>Double click to see the solution.</summary>

```python
def policy_improvement(V, env, discount_factor=0.9):
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    P = env.unwrapped.P

    policy = np.zeros((num_states, num_actions))

    for s in range(num_states):
        action_values = np.zeros(num_actions)

        for a in range(num_actions):
            for p, s_, r, _ in P[s][a]:
                action_values[a] += p * (r + discount_factor * V[s_])

        # Greedy action (or break ties arbitrarily)
        best_action = np.argmax(action_values)

        # Deterministic policy: all prob on best_action
        policy[s, best_action] = 1.0

    return policy
````

In [ ]:
policy = policy_improvement(V_pi, env.env)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7, 7), constrained_layout=True)

ax[0].imshow(frames[0])
ax[0].axis('off')
plot_policy_and_values(V_pi, policy, env, ax[1])
plt.show()

**Question**: What is the policy to follow in the states next to the holes? Why?

<details>
<summary>Double click to see the solution.</summary>

The agent will learn that is safer to move in the perpendicular direction to the intended direction, which succeeds with probability 2/3, rather than moving in the intended direction, which succeeds with probability 1/3. This is because the agent can slip and fall into a hole if it moves in the intended direction.

### Exercise 1.3: Implement policy iteration

Now it's time to implement the **policy iteration** algorithm. The policy iteration algorithm consists of two main steps: policy evaluation and policy improvement. We will iterate between these two steps until the policy converges.

Use the previously defined `policy_evaluation` and `policy_improvement` functions to implement the policy iteration algorithm.

In [ ]:
def policy_iteration(env, discount_factor=0.9, theta=1e-6, max_iterations=100):
    n_states = env.observation_space.n
    n_actions = env.action_space.n

    # Start with uniform random policy
    policy = np.ones((n_states, n_actions)) / n_actions

    for i in range(max_iterations):
        raise NotImplementedError("Comment out this line and and implement the policy iteration algorithm.")
        V = ...
        new_policy = ...

        if np.array_equal(policy, new_policy):
            print(f"Policy converged after {i + 1} iterations.")
            break

        policy = new_policy

    return policy, V

<details>
<summary>Double click to see the solution.</summary>

```python
def policy_iteration(env, discount_factor=0.9, theta=1e-6, max_iterations=100):
    n_states = env.observation_space.n
    n_actions = env.action_space.n

    # Start with uniform random policy
    policy = np.ones((n_states, n_actions)) / n_actions

    for i in range(max_iterations):
        V = policy_evaluation(policy, env, discount_factor=discount_factor, theta=theta)
        new_policy = policy_improvement(V, env, discount_factor=discount_factor)

        if np.array_equal(policy, new_policy):
            print(f"Policy converged after {i + 1} iterations.")
            break

        policy = new_policy

    return policy, V

````

In [ ]:
optimal_policy, V_pi = policy_iteration(env.env, discount_factor=0.9, theta=1e-6, max_iterations=100)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7, 7), constrained_layout=True)

ax[0].imshow(frames[0])
ax[0].axis('off')

plot_policy_and_values(V_pi, policy, env, ax[1])

plt.show()

In [ ]:
observation, info = env.reset(seed=seed)
env.action_space.seed(seed)

In [ ]:
done = False
total_reward = 0
frames = []

while not done:
    # Save the frame for the video
    frame = env.render()
    frames.append(frame)

    # Sample an action from the optimal policy
    action = np.argmax(optimal_policy[observation])
    
    # Take a step in the environment
    observation, reward, terminated, truncated, info = env.step(action)
    
    # Add the reward to the total reward
    total_reward += reward

    # Check if the episode has ended
    done = terminated or truncated

frame = env.render()
frames.append(frame)

print("Episode finished. Total reward:", total_reward)
env.close()

In [ ]:
optimal_video_path = os.path.join('videos', f'frozenlake_optimal_slippery_{is_slippery}.mp4')
imageio.mimsave(optimal_video_path, frames, fps=5)

In [ ]:
Video(optimal_video_path, width=400)

We have implemented policy evaluation and policy iteration algorithms to find the optimal policy for the FrozenLake environment. We have implemented an iterative policy evaluation algorithm that computes the value function for a given policy, and then used this to extract the optimal policy.

**Question**: Go back to the beginning of the notebook and change `is_slippery` to `False`. How does the optimal policy change? How do the state values following the optimal policy change?

<details>
<summary>Double click to see the solution.</summary>

The optimal policy differs between the two settings. When `is_slippery=True`, the agent’s moves become stochastic, so it often chooses actions perpendicular to the intended direction (which succeed with probability 2/3) to avoid accidentally slipping into a hole. You’ll also notice that, under this policy, states far from the goal have values near zero—because reaching the goal requires many risky steps, each discounted by γ.
On the other hand, when `is_slippery=False`, transitions are deterministic and the agent simply follows the shortest safe path to the goal. As a result, even the initial states attain high values, since the agent can reliably reach the goal in only a few steps.

### Summary

- Our objective is to learn a policy $\pi$ (a way of behaving) that maximizes the reward over time, the **return**.

- First we need to know how valuable are certain states and actions. To do so we have two different **value functions**:

    - The **state-value** function $v_{\pi}(s)$ tells  us the expected reward we will get if we start in a given state $s$ and then follow policy $\pi$.
    - The **action-value** function $q_{\pi}(s, a)$ tells us the expected reward we will get if we start in a give state $s$, select a given action $a$, and then follow a given policy $\pi$.

- We can use the **Bellman expectation** equations for **policy evaluation** to calculate the **state-values** or the **action-values** of a MDP. These equations are linear, so can be solved exactly for small MDPs. However, for large MDPs, we use iterative methods, as we did for the Frozen Lake problem

- We can extract a policy after **policy evaluation** by acting greedily, that is, to select the actions that lead us to better states. This is step is known as **policy improvement**.

- To get the optimal policy, we can use the **Bellman optimality** equations, to calculate the optimal **state-values** or the optimal **action-values**. If we knew them, we only need to act greedily to find the optimal policy. However, these equations are nonlinear and there's no fast way of solving them.

- Alternatively, we can do **policy iteration**, which consists on iterating **policy evaluation** and **policy improvement**. We are guaranteed to converge to a **optimal policy**.